<a href="https://colab.research.google.com/github/Lomzem/LomzemStockQuant/blob/main/LomzemDailyPolygon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
holiday_additions = [datetime.date(2022, 4, 15)]

In [ ]:
import requests
import holidays
import os
import pandas as pd
import sys

def get_stock_data(date, raw_stock_data_path):
	api_url = f'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?adjusted=true&apiKey=IyEWxobQmnBJh0QwQeNOWH7QggrJGFjp'
	try:
		data = requests.get(api_url).json()
		stock_df = pd.DataFrame(data['results'])
		stock_df.rename(columns = {'T': 'Ticker', 'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume', 'n': 'Trades', 'vw': 'VWAP'}, inplace=True)
		stock_df.insert(1, 'Date', date)
		stock_df.to_csv(f'{raw_stock_data_path}/{date}-allstockdaily.csv', index=False)
		print(f'Got {raw_stock_data_path}/{date}-allstockdaily.csv')
	except KeyError:
		print('Wait a little longer', date)
		# sys.exit()

def get_needed_days():
	raw_stock_data_path = '/content/drive/MyDrive/LomzemDailyPolygon/rawstockdata'
	oldest_date = datetime.date(2022, 3, 1)
	# current_day = datetime.datetime.today().date()
	current_day = datetime.date(2022, 5, 3)
	# files = [datetime.date(int(file.split('-')[0]), int(file.split('-')[1]), int(file.split('-')[2])) for file in os.listdir(raw_stock_data_path)]
	# files = [datetime.date(int(file.split('-')[0]), int(file.split('-')[1]), int(file.split('-')[2])) for file in os.listdir(raw_stock_data_path)]
	while current_day >= oldest_date:
		while current_day.weekday() not in set([5, 6]) and current_day not in holidays.US() and current_day not in holiday_additions and current_day not in [datetime.date(int(file.split('-')[0]), int(file.split('-')[1]), int(file.split('-')[2])) for file in os.listdir(raw_stock_data_path)]: 
			get_stock_data(current_day, raw_stock_data_path)
		current_day += datetime.timedelta(days=-1)

get_needed_days()
# get_stock_data(datetime.date(2022, 5, 2), '/content/drive/MyDrive/LomzemDailyPolygon/rawstockdata')

NameError: ignored

In [ ]:
import pandas as pd
import os

cum_stock_data = pd.DataFrame()

for file in os.listdir('/content/drive/MyDrive/LomzemDailyPolygon/rawstockdata'):
	df = pd.read_csv(f'/content/drive/MyDrive/LomzemDailyPolygon/rawstockdata/{file}')
	cum_stock_data = pd.concat([cum_stock_data, df])
 
cum_stock_data.sort_values(['Ticker', 'Date'], inplace=True)
cum_stock_data['Gap %'] = (cum_stock_data['Open'] / cum_stock_data.groupby('Ticker').shift(1)['Close'] - 1) * 100
cum_stock_data['Range %'] = (cum_stock_data['High'] / cum_stock_data['Low'] - 1) * 100
cum_stock_data['Gap %'] = (cum_stock_data['Open'] / cum_stock_data.groupby('Ticker').shift(1)['Close'] - 1) * 100
cum_stock_data['closedRed'] = (cum_stock_data['Close'] < cum_stock_data['Open'])
cum_stock_data = cum_stock_data.fillna(0)

print(cum_stock_data)
df = cum_stock_data

     Ticker        Date     Volume      VWAP    Open   Close      High  \
4702      A  2022-03-01  2634071.0  131.7636  130.83  131.93  132.9150   
5028      A  2022-03-02  1863110.0  133.1516  132.68  133.16  133.9600   
4008      A  2022-03-03  3592139.0  136.4757  133.73  137.17  138.0000   
4544      A  2022-03-04  3848511.0  134.2259  135.85  133.89  137.1200   
4714      A  2022-03-07  2376504.0  130.3950  133.55  130.34  133.5500   
...     ...         ...        ...       ...     ...     ...       ...   
7301   ZYXI  2022-04-27   393331.0    7.0662    6.89    6.97    7.3000   
1849   ZYXI  2022-04-28   437947.0    6.9990    6.95    7.14    7.2050   
38     ZYXI  2022-04-29   806207.0    6.6018    6.39    6.37    7.1000   
6535   ZYXI  2022-05-02   654142.0    6.0873    6.31    6.18    6.3363   
196    ZYXI  2022-05-03   440316.0    6.6108    6.56    6.52    6.8300   

           Low              t   Trades      Gap %    Range %  closedRed  
4702  130.0850  1646168400000  41423.

In [ ]:
import requests
import os
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
df.sort_values(['Ticker', 'Date'], inplace=True)

display(df.loc[(df['High']/df['Low']-1>.2) & (df['High']>1) & (df['Trades']>10000)])
# display(df.loc[(df['High'].shift(1)>1) & (df['Volume'].shift(1)>40000000) & (df['High'].shift(1)/df['Open'].shift(1)-1>.3) & (df['Low'].shift(1)<30)])
# display(df)
# ddf = pd.concat([ddf, df[['Ticker', 'FTime']]])
# ddf['FTime'] = ddf['FTime'].dt.date

# # df.loc[192]


,Ticker,Date,Volume,VWAP,Open,Close,High,Low,t,Trades,Gap %,Range %,closedRed
1782,ABSI,2022-03-14,943139.0,8.0684,8.970,7.48,9.1000,7.40,1647288000000,13181.0,-0.554324,22.972973,True
4070,ACCD,2022-04-28,2819548.0,10.4007,12.270,11.06,12.4800,9.88,1651176000000,29883.0,-0.081433,26.315789,True
1024,ACCD,2022-04-29,22452587.0,5.6382,6.305,5.56,6.4800,4.61,1651262400000,118443.0,-42.992767,40.563991,True
5119,ACCD,2022-05-02,18899300.0,6.6778,5.575,6.43,7.2300,5.55,1651521600000,97269.0,0.269784,30.270270,False
3709,ACER,2022-04-04,20703688.0,3.4126,3.640,3.04,3.7700,2.93,1649102400000,73657.0,24.657534,28.668942,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,ZY,2022-04-19,3282789.0,1.7986,1.660,2.01,2.0300,1.59,1650398400000,13460.0,1.219512,27.672956,False
1603,ZY,2022-04-25,2878146.0,1.9606,1.780,2.05,2.1408,1.74,1650916800000,13732.0,0.000000,23.034483,False
3541,ZY,2022-04-26,2505825.0,1.7679,2.030,1.71,2.1031,1.69,1651003200000,12469.0,-0.975610,24.443787,True
1742,ZYME,2022-04-29,36641405.0,7.0584,7.190,6.11,7.8800,6.08,1651262400000,174685.0,44.959677,29.605263,True


In [ ]:
import os
import requests
import pandas as pd
import datetime
import holidays

def get_stock_data_i(date, ticker):
	api_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey=IyEWxobQmnBJh0QwQeNOWH7QggrJGFjp'
	try:
		data = requests.get(api_url).json()
		stock_df = pd.DataFrame(data['results'])
		stock_df.rename(columns = {'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume', 't': 'Time'}, inplace=True)
		stock_df.insert(0, 'Ticker', ticker)
		stock_df.insert(1, 'FTime', stock_df.apply(lambda row: pd.to_datetime(row['Time']-1.44e+7, unit='ms'), axis=1))
		stock_df.to_csv(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{ticker}-{date}.csv', index=False)
		print(f'Got /content/drive/MyDrive/LomzemStockCharts/rawstockdata/{ticker}-{date}.csv')
	except KeyError:
		print('Wait a little longer', date, ticker)
		# sys.exit()

def delta_i_date(date, delta):
	if delta == -1:
		match = 0
		while match == 0:
			date = date - datetime.timedelta(days=1)
			if date.weekday() <= 4 and date not in holidays.US() and date not in holiday_additions:
				match = 1
		return date
	elif delta == 1:
		match = 0
		while match == 0:
			date = date + datetime.timedelta(days=1)
			if date.weekday() <= 4 and date not in holidays.US() and date not in holiday_additions:
				match = 1
		return date

def str_to_date(str):
	return datetime.datetime.strptime(str, '%Y-%m-%d').date()

def get_intraday_data():
	# test = df.loc[(df['Ticker']=='AKAN') & (df['Date'].dt.date==datetime.date(2022, 3, 14))].values
	# print(test)
	df.sort_values('Date', inplace=True, ascending=False)
	noteable_gappers = df.loc[(df['Range %']>30) & (df['High']>1) & (df['Trades']>13000)]

	for index, row in noteable_gappers.iterrows():
		date = str_to_date(row['Date'])
		while not (f'{row["Ticker"]}-{date}.csv') in [file for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata')]:
			get_stock_data_i(date, row['Ticker'])
		while not (f'{row["Ticker"]}-{delta_i_date(date, -1)}.csv') in [file for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata')]:
			if len(df.loc[(df['Ticker']==row['Ticker']) & (pd.to_datetime(df['Date'])==str(delta_i_date(date, -1)))].values) == 0:
				break
			else:
				get_stock_data_i(delta_i_date(date, -1), row['Ticker'])
		if not delta_i_date(date, 1) >= datetime.datetime.today().date() - datetime.timedelta(1):
			while not (f'{row["Ticker"]}-{delta_i_date(date, 1)}.csv') in [file for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata')]:
				if len(df.loc[(df['Ticker']==row['Ticker']) & (df['Date']==str(delta_i_date(date, 1)))].values) == 0:
					break
				else:
					get_stock_data_i(delta_i_date(date, 1), row['Ticker'])

# display(df.loc[(df['Range %']>30) & (df['High']>1) & (df['Trades']>13000)])
get_intraday_data()

NameError: ignored